In [1]:
#Connect to Opticstudio
import clr, os, winreg
from itertools import islice

#READ ME
#For the following to function, you MUST have your optics file open in Optic Studio, and
#you MUST go to "Programming" and then click on "Interactive Extension", then run the
#python program.

# determine the Zemax working directory
aKey = winreg.OpenKey(winreg.ConnectRegistry(None, winreg.HKEY_CURRENT_USER), r"Software\Zemax", 0, winreg.KEY_READ)
zemaxData = winreg.QueryValueEx(aKey, 'ZemaxRoot')
NetHelper = os.path.join(os.sep, zemaxData[0], r'ZOS-API\Libraries\ZOSAPI_NetHelper.dll')
winreg.CloseKey(aKey)

# add the NetHelper DLL for locating the OpticStudio install folder
clr.AddReference(NetHelper)
import ZOSAPI_NetHelper

pathToInstall = ''
# uncomment the following line to use a specific instance of the ZOS-API assemblies
#pathToInstall = r'C:\C:\Program Files\Zemax OpticStudio'

# connect to OpticStudio
success = ZOSAPI_NetHelper.ZOSAPI_Initializer.Initialize(pathToInstall)

zemaxDir = ''
if success:
    zemaxDir = ZOSAPI_NetHelper.ZOSAPI_Initializer.GetZemaxDirectory()
    print('Found OpticStudio at:   %s' + zemaxDir)
else:
    raise Exception('Cannot find OpticStudio')

# load the ZOS-API assemblies
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI.dll'))
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI_Interfaces.dll'))
import ZOSAPI

TheConnection = ZOSAPI.ZOSAPI_Connection()
if TheConnection is None:
    raise Exception("Unable to intialize NET connection to ZOSAPI")

TheApplication = TheConnection.ConnectAsExtension(0)
if TheApplication is None:
    raise Exception("Unable to acquire ZOSAPI application")

if TheApplication.IsValidLicenseForAPI == False:
    raise Exception("License is not valid for ZOSAPI use.  Make sure you have enabled 'Programming > Interactive Extension' from the OpticStudio GUI.")

TheSystem = TheApplication.PrimarySystem
if TheSystem is None:
    raise Exception("Unable to acquire Primary system")

print('Connected to OpticStudio')
####################################################################################################

Found OpticStudio at:   %sc:\program files\ansys zemax opticstudio 2022 r2.02
Connected to OpticStudio


In [2]:
#Other Imports
import numpy as np
import sys, time

##STAR Setup
analysis = TheSystem.Analyses
tools = TheSystem.Tools
surf_inds = [3,8,13,18,23,27,31,36,40,44]
surf_names = ['M1','M2','M3','LODM','Dicro','OAP1','OAP2','BMC','OAP3','VVC']
n_surf = len(surf_inds)

# Get Surfaces
TheLDE = TheSystem.LDE
move_m2 = TheLDE.GetSurfaceAt(7)
surf_lodm = TheLDE.GetSurfaceAt(18)

#Data filepath
data_dir = 'c:\\Users\\locsst\\Desktop\\STAR_ex\\STAR_data\\' 

#Clear Any Leftover Data from previous runs
for i in range(n_surf):
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.UnloadData()

In [3]:
#Open displacement data
for i in range(n_surf):
    disp_file = data_dir + 'Surface_' + str(surf_inds[i]).zfill(2) + '_Deformation.txt'
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.SetDataIsGlobal()
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.Fits.Settings.SetAllValues(2,2,2,10,10**(-9),0.5)
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.ImportDeformations(disp_file)

    if TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.AreDeformationsApplied():
        print('Deformations Applied for: ' + surf_names[i])

Deformations Applied for: M1
Deformations Applied for: M2
Deformations Applied for: M3
Deformations Applied for: LODM
Deformations Applied for: Dicro
Deformations Applied for: OAP1
Deformations Applied for: OAP2
Deformations Applied for: BMC
Deformations Applied for: OAP3
Deformations Applied for: VVC


In [5]:
#Read out data
for i in range(n_surf):
    disp_data =  TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs
    disp_data.GetTransformValues()

    print('Rigid Body motions for ' + surf_names[i] + ':')
    print('X: ',  disp_data.Decenters.X)
    print('Y: ',  disp_data.Decenters.Y)
    print('Z: ',  disp_data.Decenters.Z)
    print('Rx: ', disp_data.Rotations.Angles.ThetaX)
    print('Ry: ', disp_data.Rotations.Angles.ThetaY)
    print('Rz: ', disp_data.Rotations.Angles.ThetaZ,'\n')

Rigid Body motions for M1:
X:  -0.00877997390717207
Y:  0.006443278495588345
Z:  -0.0001878172407143719
Rx:  0.009330373791429562
Ry:  0.010909065241248574
Rz:  0.0010874729662521028 

Rigid Body motions for M2:
X:  -0.026012407642935942
Y:  0.03433086339845648
Z:  0.04865396879968614
Rx:  0.027590557268282318
Ry:  0.007659468576360001
Rz:  0.008824934513622318 

Rigid Body motions for M3:
X:  -0.011562229865697882
Y:  0.01925769677212402
Z:  0.005310221535937368
Rx:  0.011862994427405718
Ry:  0.01692316629128452
Rz:  -0.0006561659624187214 

Rigid Body motions for LODM:
X:  -0.014150496408346308
Y:  0.020587422544260412
Z:  0.0024354622738729207
Rx:  0.01143594342497726
Ry:  0.016938661904609183
Rz:  0.0011769685741368546 

Rigid Body motions for Dicro:
X:  -0.012587319767692295
Y:  0.019609140611863078
Z:  0.0027860412188590333
Rx:  0.01172113499301577
Ry:  0.016928559201749373
Rz:  0.0007845656520540725 

Rigid Body motions for OAP1:
X:  -0.012366566543628554
Y:  0.02017708321447332